In [35]:
import re
import time
import pyperclip

def extract_values(text: str) -> str:
    # 1. Split into lines, drop empty ones
    lines = [ln for ln in text.splitlines() if ln.strip()]

    # 2. For each line, remove everything up to and including the first “:”
    #    then normalize non-numeric entries to “-”
    values = []
    for ln in lines:
        data = re.sub(r'^.*?:\s*', '', ln)
        if not re.search(r'\d', data):
            data = '-'
        values.append(data)

    # 3. Join as a comma-separated string and adjust if it contains “/”
    result = ','.join(values)
    if "/" in result:
        result = "," + result + ",-"
    return result

def watch_clipboard(poll_interval: float = 0.5):
    last = None
    while True:
        current = pyperclip.paste()
        if current != last:
            transformed = extract_values(current)
            pyperclip.copy(transformed)
            print(f"Clipboard updated → {transformed}")
            last = transformed
        time.sleep(poll_interval)

if __name__ == "__main__":
    print("Watching clipboard…  (Ctrl-C to quit)")
    watch_clipboard()


Watching clipboard…  (Ctrl-C to quit)
Clipboard updated → ,-,-,-,-,    # 1. Split into lines, drop empty ones,-,-,-,-,-,-,-,-,-,    # 3. Join as a comma-separated string and adjust if it contains “/”,-,-,-,-,float = 0.5):,-,-,-,-,-,-,-,-,-,-,-,-,-
Clipboard updated → 10,35,10,10,20,40,0,40
Clipboard updated → ,154 / 252 / 542 / 999,154 / 252 / 542 / 999,252 / 542 / 999,252 / 542 / 999,154 / 252 / 542 / 999,-,-
Clipboard updated → 10,35,10,10,40,80,40,40
Clipboard updated → ,252 / 542 / 999,252 / 542 / 999,252 / 542 / 999,542 / 999,112 / 154 / 252 / 542 / 999,-,-
Clipboard updated → 10,10,10,10,40,40,0,20
Clipboard updated → ,154 / 252 / 542 / 999,154 / 252 / 542 / 999,252 / 542 / 999,252 / 542 / 999,154 / 252 / 542 / 999,-,-
Clipboard updated → 10,10,-40,35,20,20,40,-40
Clipboard updated → ,-,253 / 542 / 999,-,-,-,-,-
Clipboard updated → 10,35,0,35,20,40,40,80
Clipboard updated → -,-,-,-,-,-
Clipboard updated → 10,35,10,10,20,40,40,20
Clipboard updated → ,154 / 252 / 542 / 999,154 / 25

KeyboardInterrupt: 

In [2]:
import re
import pyperclip

def extract_values(text):
    # 1. Split into lines, drop empty ones
    lines = [ln for ln in text.splitlines() if ln.strip()]

    # 2. For each line, remove everything up to and including the first “:”
    #    then normalize non-numeric entries to “-”
    values = []
    for ln in lines:
        data = re.sub(r'^.*?:\s*', '', ln)
        # if it’s not digits/slashes, treat as “-”
        if not re.search(r'\d', data):
            data = '-'
        values.append(data)

    # 3. Join as a comma-separated string
    result = ','.join(values)
    if ("/") in result:
        result = "," + result + ",-"
    print(result)
    pyperclip.copy(result)

In [34]:
text = """



    poison status effect elden ring wiki guide 25pxPoison: 542 / 999
    scarlet rot status effect elden ring wiki guide 25pxScarlet Rot: 542 / 999
    hemorrhage status effect elden ring nightreign wiki guide 100pxBlood Loss: 542 / 999
    frostbite status effect elden ring wiki guide 25pxFrostbite: 542 / 999

    sleep status effect elden ring wiki guide 25pxSleep: Immune
    madness status effect elden ring wiki guide 25pxMadness: Immune


"""

extract_values(text)

,542 / 999,542 / 999,542 / 999,542 / 999,-,-,-
